## 라이브러리, 토큰화된 로우 호출

In [2]:
import pandas as pd 
import numpy as np 

In [ ]:
!pip install glove_python_binary

In [3]:
import pickle #토큰화 완료한 리스트 불러오기 위해 저장(여러형식) 라이브러리 pickle 이용 

In [4]:
with open("data/all_tokens.pickle","rb") as f:
    token_list= pickle.load(f)

In [5]:
len(token_list) #샘플리뷰 수와 일치 

1564713

## Glove Embedding ( glove_model2 )

글로브 모델과 워드투백 모델 중, 가짜 리뷰선별에 적합한 모델을 선정하기 위해 두 모델 모두 실행해 보았다. 
그 中 글로브 임베딩을 실행한 코드 

In [ ]:
! pip install glove_python_binary

In [6]:
from glove import Corpus, Glove

In [7]:
corpus=Corpus()

In [9]:
import random

In [15]:
token_list=random.sample(token_list,5)

#예시로 샘플 5개만 출력해봄 

In [16]:
corpus.fit(token_list, window=8) #백터 계산 시, 주변 8개 단어까지 영향범위로 설정

In [17]:
# model
glove = Glove(no_components=128, learning_rate=0.01) #128차원으로 설정
glove.fit(corpus.matrix, epochs=50, no_threads=4, verbose=False) #epoch:학습횟수 
glove.add_dictionary(corpus.dictionary)

In [20]:
glove.save('glove_model2.model')

In [21]:
glove_model=Glove.load('glove_model2.model')

In [22]:
glove.word_vectors.shape

(29, 128)

In [23]:
word_dict = {}
for word in  glove_model.dictionary.keys():
    word_dict[word] = glove_model.word_vectors[glove_model.dictionary[word]]
print('[Success !] Lengh of word dict... : ', len(word_dict))

[Success !] Lengh of word dict... :  29


In [24]:
glove_model.dictionary.keys() 

dict_keys(['미모', '찾아서', '도리', '최고', '입니다', '눈뽕', '귀', '뽕', '빼고', '남는게', '없는', '영화', '트랜스포머', '동', '급', '새해', '알리는', '망작', '리얼', '염력', '피', '했지만', '리틀', '포레스트', '피하', '했다', '지루한', 'ㅠㅠ', '딴거볼껄'])

In [25]:
len(glove_model.word_vectors) #128차원의 백터로 임베딩된 단어들 확인 

29

## 문장 단위 임베딩

In [26]:
def sent2vec_glove(token_list, embedding_dim=128):
  
    size = len(token_list)
    matrix = np.zeros((size, embedding_dim))
    word_table = word_dict     # glove word_dict

    for i, token in enumerate(token_list):
        vector = np.array([
            word_table[t] for t in token
            if t in word_table
        ])

        if vector.size != 0:
            final_vector = np.mean(vector, axis=0)
            matrix[i] = final_vector

    return matrix

+ 문장 임베딩 값을 kmeans군집화 후 TSNE 시각화 

In [27]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
sentence_glove = sent2vec_glove(token_list)
sentence_glove.shape

(5, 128)

In [ ]:
# clustering
k = 3
kmeans = KMeans(n_clusters=k, random_state=2022)
y_pred = kmeans.fit_predict(sentence_glove)

# tsne
tsne = TSNE(verbose=1, perplexity=100, random_state=2022)     # perplexity : 유사정도
X_embedded = tsne.fit_transform(sentence_glove)
print('Embedding shape 확인', X_embedded.shape)

# 시각화
sns.set(rc={'figure.figsize':(20,20)})
# colors
palette = sns.hls_palette(10, l=.4, s=.9) 
# plot
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y_pred,
                legend='full',palette='bright')     # kmeans로 예측

plt.title('t-SNE with KMeans Labels and Glove Embedding')
plt.savefig("t-sne_question_glove_embedding.png")
plt.show()